# Objective of the Project
The objective of the project is to be able to open DBC files, the standard format of SINAN, and transform them into a table, converting the encodings according to the data dictionary.

It will analyze notifications of accidents involving venomous animals, which are made available by DATASUS through the link https://datasus.saude.gov.br/acesso-a-informacao/doencas-e-agravos-de-notificacao-de-2007-em-diante-sinan/, where there are several databases available.

At the end of the project, there will be a data transformation and analysis notebook that can be adapted for other files available from SINAN.

# Importing Libraries
After installing the library, we need to import the one we will use:
*   Pandas: Standard library for data 
analysis in Python.

In [ ]:
import pandas as pd

**The cell below is necessary to allow us to visualize the entire table on the screen:**

In [ ]:
pd.options.display.max_columns = None

# Searching for and accessing the project's DBC file.
After downloading the DBC file, we will use the TabWin program to transform the data into CSV format, which is more commonly used and easier to work with.

The TabWin program can be downloaded from the following link: https://datasus.saude.gov.br/transferencia-de-arquivos/

Select:
*   Source: Applications - *TABWIN/TABNET* - Tools for data tabulation
*   Modality: Programs
*   File Type: *TABWIN* data tabulator for Windows

Click on **Enviar** (Submit) to upload the file. Once uploaded, click on **Download**. The website will reload, and a link named "arquivo.zip" will be downloaded.






The Pandas library will be used to transform the CSV file into a readable table:

In [ ]:
datas = pd.read_csv('/ANIMBR17.csv', encoding='latin1')

To visualize the transformation made, you can use the following command:

In [ ]:
datas

# Conversion Based on SINAN Data Dictionary
In this section, we will define which columns to keep or exclude.

For this task, we need to analyze the data dictionary available at the following links:
*   Individual Notification Data:  http://portalsinan.saude.gov.br/images/documentos/Agravos/Notificacao_Individual/DIC_DADOS_NET---Notificao-Individual_rev.pdf
*   Accidents with Venomous Animals Data:
http://portalsinan.saude.gov.br/images/documentos/Agravos/AAP/DIC_DADOS_Animais_Pedonhentos_v5.pdf

To visualize the columns of the created table, you can use the command **.columns**

In [ ]:
datas.columns

Among the columns above, we need to choose which ones to keep and pass their names to a list.

In [ ]:
columns_to_maintain =  ['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
       'ID_MUNICIP', 'ID_REGIONA', 'DT_SIN_PRI', 'SEM_PRI', 'ANO_NASC',
       'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF',
       'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS', 'DT_INVEST', 'ID_OCUPA_N',
       'ANT_DT_ACI', 'ANT_UF', 'ANT_MUNIC_', 'ANT_LOCALI', 'ANT_TEMPO_',
       'ANT_LOCA_1', 'MCLI_LOCAL', 'CLI_DOR', 'CLI_EDEMA', 'CLI_EQUIMO',
       'CLI_NECROS', 'CLI_LOCAL_', 'CLI_LOCA_1', 'MCLI_SIST', 'CLI_NEURO',
       'CLI_HEMORR', 'CLI_VAGAIS', 'CLI_MIOLIT', 'CLI_RENAL', 'CLI_OUTR_2',
       'CLI_OUTR_3', 'CLI_TEMPO_', 'TP_ACIDENT', 'ANI_TIPO_1', 'ANI_SERPEN',
       'ANI_ARANHA', 'ANI_LAGART', 'TRA_CLASSI', 'CON_SOROTE', 'NU_AMPOLAS',
       'NU_AMPOL_1', 'NU_AMPOL_8', 'NU_AMPOL_6', 'NU_AMPOL_4', 'NU_AMPO_7',
       'NU_AMPO_5', 'NU_AMPOL_9', 'NU_AMPOL_3', 'COM_LOC', 'COM_SECUND',
       'COM_NECROS', 'COM_COMPOR', 'COM_DEFICT', 'COM_APUTAC', 'COM_SISTEM',
       'COM_RENAL', 'COM_EDEMA', 'COM_SEPTIC', 'COM_CHOQUE', 'DOENCA_TRA',
       'EVOLUCAO', 'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA']

Thus, we use this list within square brackets, next to the name of our table. We can assign a new name to the table or use the same name. In the latter option, we replace our dataframe, while in the former, we create a new one.

In [ ]:
datas = datas[columns_to_maintain].reset_index(drop=True)

Let's see how it turned out

In [ ]:
datas

# Data Recoding
One of the major issues with DBF files is the encoding of variables. Simply opening the file does not provide clear information, as we have to refer to the data dictionary of that file. With Python, we can recode the columns by writing the data dictionary for each one. It is a time-consuming task, but we only need to do it once, and then it will be ready.

Upon evaluating the data dictionary, we notice that many variables have values such as YES and NO. For these cases, we will create a mini dictionary and apply the .replace function, which will replace the values found in the table with those seen in the dictionary.

Let's take an example column and then apply it to the others.

In [ ]:
datas['MCLI_LOCAL'] = datas['MCLI_LOCAL'].replace({1:'Sim', 2: 'Não', 9:'Ignorado'})

Let's see how it turned out:

In [ ]:
datas['MCLI_LOCAL']

Now we will create a list with the columns that have the values **YES** and **NO**.

In [ ]:
columns_yes_or_no_type = ['MCLI_LOCAL', 'CLI_DOR', 'CLI_EDEMA', 'CLI_EQUIMO', 
                          'CLI_NECROS', 'CLI_LOCAL_', 'MCLI_SIST', 'CLI_NEURO',
                          'CLI_HEMORR', 'CLI_VAGAIS', 'CLI_MIOLIT', 'CLI_RENAL', 
                          'CLI_OUTR_2', 'CON_SOROTE', 'COM_LOC', 'COM_SECUND', 
                          'COM_NECROS', 'COM_COMPOR',  'COM_DEFICT', 'COM_APUTAC',
                          'COM_SISTEM', 'COM_RENAL',  'COM_EDEMA', 'COM_SEPTIC', 
                          'COM_CHOQUE', 'DOENCA_TRA']

Now we will perform a bulk change of this data. For that, we will use the **for** loop, which allows us to apply the same operation multiple times, only changing one parameter.

In [ ]:
for column in columns_yes_or_no_type:
  datas[column] = datas[column].replace({1:'Sim', 2: 'Não', 9:'Ignorado'})

datas

Some fields are of the date type. Let's identify them using the **.info()** function.

In [ ]:
datas.info()

With the identified fields, let's now add them to a list.

In [ ]:
columns_date_type = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_INVEST', 
                     'DT_OBITO', 'DT_ENCERRA', 'DT_DIGITA', 'ANT_DT_ACI']

Again, we can use the **for** loop to make changes in these fields. To convert the date type to the desired format, we should use the pandas function **.to_datetime**, where we pass the column name, the desired date format, and error handling as parameters.

In [ ]:
for column in columns_date_type:
  datas[column] = pd.to_datetime(datas[column], format='%Y%m%d', errors='coerce')

Let's check how one of the fields turned out.

In [ ]:
datas['DT_NOTIFIC']

Now comes the most labor-intensive part, where we have to rewrite all the codifications from the data dictionary. In other words, we will create multiple dictionaries based on the SINAN dictionary.

In [ ]:
# ANT_TEMPO_
datas['ANT_TEMPO_'] = datas['ANT_TEMPO_'].replace({1: '0 - 1h',
                                                   2: '1 - 3h',
                                                   3: '3 - 6h',
                                                   4: '6 - 12h',
                                                   5: '12 e 24h',
                                                   6: '24 e +h',
                                                   9: 'Ignorado'})

# ANT_LOCA_1
datas['ANT_LOCA_1'] = datas['ANT_LOCA_1'].replace({1: 'Cabeça',
                                                   2: 'Braço',
                                                   3: 'Ante-Braço',
                                                   4: 'Mão',
                                                   5: 'Dedo da Mão',
                                                   6: 'Tronco',
                                                   7: 'Coxa',
                                                   8: 'Perna',
                                                   9: 'Pé',
                                                   10: 'Dedo do Pé',
                                                   99: 'Ignorado'})

# CLI_TEMPO_
datas['CLI_TEMPO_'] = datas['CLI_TEMPO_'].replace({1: 'Normal',
                                                   2: 'Alterado',
                                                   9: 'Não Realizado'})

# TP_ACIDENT
datas['TP_ACIDENT'] = datas['TP_ACIDENT'].replace({1: 'Serpente',
                                                   2: 'Aranha',
                                                   3: 'Escorpião',
                                                   4: 'Lagarta',
                                                   5: 'Abelha',
                                                   6: 'Outros',
                                                   9: 'Ignorado'})

# ANI_SERPEN
datas['ANI_SERPEN'] = datas['ANI_SERPEN'].replace({1: 'Botrópico',
                                                   2: 'Crotálico',
                                                   3: 'Elapídico',
                                                   4: 'Laquético',
                                                   5: 'Serpente Não Peçonhenta',
                                                   9: 'Ignorado'})

# ANI_ARANHA
datas['ANI_ARANHA'] = datas['ANI_ARANHA'].replace({1: 'Foneutrismo',
                                                   2: 'Loxoscelismo',
                                                   3: 'Latrodectismo',
                                                   4: 'Outra Aranha',
                                                   9: 'Ignorado'})

# ANI_LAGART
datas['ANI_LAGART'] = datas['ANI_LAGART'].replace({1: 'Lonomia',
                                                   2: 'Outra Lagarta',
                                                   9: 'Ignorado'})

# TRA_CLASSI
datas['TRA_CLASSI'] = datas['TRA_CLASSI'].replace({1: 'Leve',
                                                   2: 'Moderado',
                                                   3: 'Grave',
                                                   9: 'Ignorado'})

# EVOLUCAO
datas['EVOLUCAO'] = datas['EVOLUCAO'].replace({1: 'Cura',
                                               2: 'Óbito por Acidente por Animais Peçonhentos',
                                               3: 'Óbito por Outras Causas',
                                               9: 'Ignorado'})
# CS_GESTANT
datas['CS_GESTANT'] = datas['CS_GESTANT'].replace({1: '1º Trimestre',
                                                   2: '2º Trimestre',
                                                   3: '3º Trimestre',
                                                   4: 'Idade Gestacional Ignorada',
                                                   5: 'Não',
                                                   6: 'Não Se Aplica',
                                                   9: 'Ignorado'})

# CS_RACA
datas['CS_RACA'] = datas['CS_RACA'].replace({1: 'Branca',
                                             2: 'Preta',
                                             3: 'Amarela',
                                             4: 'Parda',
                                             5: 'Indígena',
                                             9: 'Ignorado'})

# CS_ESCOL_N
datas['CS_ESCOL_N'] = datas['CS_ESCOL_N'].replace({1: '1ª a 4ª Série Incompleta do EF',
                                                   2: '4ª Série Completa do EF (antigo 1° grau)',
                                                   3: '5ª à 8ª Série Incompleta do EF (Antigo Ginásio ou 1° grau)',
                                                   4: 'Ensino fundamental Completo (Antigo Ginásio ou 1° grau)',
                                                   5: 'Ensino Médio Incompleto (Antigo Colegial ou 2° grau)',
                                                   6: 'Ensino Médio Completo (Antigo Colegial ou 2° grau)',
                                                   7: 'Educação Superior Incompleta',
                                                   8: 'Educação Superior Completa',
                                                   9: 'Ignorado',
                                                   10: 'Não se Aplica'})


The **AGE** column is a bit different. SINAN uses a decoding format for this column as follows: First digit:
<br>1: Hour
<br>2: Day
<br>3: Month
<br>4: Year

Examples:
<br>3009: Nine months
<br>4018: Eighteen years
<br>2150: 150 days

Since we are only interested in age in years, we will use the following logic to convert the values in the column to years:

*   If the value is greater than 4000, subtract 4000 from the age to obtain the converted value.
*   If the age is less than 4000, assign age 0.

In [ ]:
ages = []

for age in datas['NU_IDADE_N']:
  if age > 4000:
    ages.append(age - 4000)
  else:
    ages.append(0)

datas['NU_IDADE_N'] = ages

# IBGE code for municipality name

Similarly to how we used a dictionary and the *replace* function to change elements in the DataFrame, we will do the same to transform IBGE codes into municipality names using the *map* function.

Firstly, we need a table where this conversion is already prepared. You can find it here:
https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/populacao%20ibge%206%20municipio%20br.csv

Let's create a DataFrame to store this data:

In [76]:
municipalities = pd.read_csv('https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/populacao%20ibge%206%20municipio%20br.csv')

Let's take a look at the created table.

In [ ]:
municipalities

Let's limit the fields of interest, just like we did before.

In [78]:
columns_to_maintain = ['IBGE6', 'Municipio']
municipalities = municipalities[columns_to_maintain]

We should index the IBGE6 field because it contains the code that we will use to relate to our main DataFrame. To do it we must use **.set_index()** function

In [79]:
municipalities = municipalities.set_index('IBGE6')

Let's create a dictionary from this table so that we can use it as a basis for transforming the main DataFrame. TO do it we're gonna use the **.to_dict()** function.

In [ ]:
municipalities = municipalities.to_dict()

Let's take a look at the dictionary to make sure everything is correct.

In [ ]:
municipalities

As we can see, there is a dictionary within another dictionary. To resolve this, we just need to pass the name of the inner dictionary as a parameter when creating the dictionary.

In [ ]:
municipalities = municipalities.to_dict()['Municipio']

To transform the data, we will use the .map function. The .map function is a function that, if the value is not found in the dictionary, it will be transformed into NaN, resulting in a loss of that data. However, it is 60 times faster than the .replace function, so we save time, especially considering that we are working with a dictionary with more than 5000 values.

Thus, we can convert the 3 columns that contain IBGE municipality codes (ID_MUNICIP, ID_MN_RESI, ANT_MUNIC_) using the dictionaries.

In [ ]:
datas['ID_MUNICIP_NOME'] = datas['ID_MUNICIP'].map(municipalities)
datas['ID_MN_RESI_NOME'] = datas['ID_MN_RESI'].map(municipalities)
datas['ANT_MUNIC_NOME'] = datas['ANT_MUNIC_'].map(municipalities)

In the same way, it is possible to change the encoding of the OCCUPATION column. Here is the link that provides access to the occupation data: https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/CBO2002%20-%20Ocupacao.csv

In [ ]:
ocupations = pd.read_csv('https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/CBO2002%20-%20Ocupacao.csv')
ocupations = ocupations.set_index('CODIGO')
ocupations = ocupations.to_dict()['TITULO']

Let's convert the column that represents occupation using IDs.

In [ ]:
datas['ID_OCUPA_N_NOME'] = datas['ID_OCUPA_N'].map(ocupations)

To conclude, let's modify the encoding of the Federative Unit to its respective abbreviation, as we did previously. Here is the link that provides access to the federative units data: https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/cod_uf.csv

In [ ]:
federative_units = pd.read_csv('https://raw.githubusercontent.com/andrejarenkow/Curso_SINAN_Udemy/main/dados/cod_uf.csv')
federative_units = federative_units.set_index('Código UF')
federative_units = federative_units.to_dict()['UF']

Let's convert the column that represents federative units using IDs.


In [ ]:
datas['SG_UF_NOME'] = datas['SG_UF'].map(federative_units)
datas['SG_UF_NOT_NOME'] = datas['SG_UF_NOT'].map(federative_units)
datas['ANT_UF_NOME'] = datas['ANT_UF'].map(federative_units)

# Renaming Columns
Another common issue we often encounter when visualizing a SINAN DBF file is the column names, which are encoded differently. Therefore, we will rename them using a function similar to what we did before.

First, let's collect the names of all the columns

In [82]:
datas.columns

Index(['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP',
       'ID_REGIONA', 'DT_SIN_PRI', 'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N',
       'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI',
       'ID_RG_RESI', 'ID_PAIS', 'DT_INVEST', 'ID_OCUPA_N', 'ANT_DT_ACI',
       'ANT_UF', 'ANT_MUNIC_', 'ANT_LOCALI', 'ANT_TEMPO_', 'ANT_LOCA_1',
       'MCLI_LOCAL', 'CLI_DOR', 'CLI_EDEMA', 'CLI_EQUIMO', 'CLI_NECROS',
       'CLI_LOCAL_', 'CLI_LOCA_1', 'MCLI_SIST', 'CLI_NEURO', 'CLI_HEMORR',
       'CLI_VAGAIS', 'CLI_MIOLIT', 'CLI_RENAL', 'CLI_OUTR_2', 'CLI_OUTR_3',
       'CLI_TEMPO_', 'TP_ACIDENT', 'ANI_TIPO_1', 'ANI_SERPEN', 'ANI_ARANHA',
       'ANI_LAGART', 'TRA_CLASSI', 'CON_SOROTE', 'NU_AMPOLAS', 'NU_AMPOL_1',
       'NU_AMPOL_8', 'NU_AMPOL_6', 'NU_AMPOL_4', 'NU_AMPO_7', 'NU_AMPO_5',
       'NU_AMPOL_9', 'NU_AMPOL_3', 'COM_LOC', 'COM_SECUND', 'COM_NECROS',
       'COM_COMPOR', 'COM_DEFICT', 'COM_APUTAC', 'COM_SISTEM', 'COM_RENAL',
       'COM_EDEMA', 'COM_SEPTI

Now let's copy this list, and we'll adapt it to create a dictionary

In [ ]:
columns_renamed = {'DT_NOTIFIC': 'Data da Notificação', 
                   'SEM_NOT': 'Semana Epidemiológica da Notificação', 
                   'NU_ANO': 'Ano da notificação ', 
                   'SG_UF_NOT': 'UF de Notificação', 
                   'ID_MUNICIP': 'Município de Notificação',
                   'ID_REGIONA': 'Código Regional da Notificação', 
                   'DT_SIN_PRI': 'Data dos Primeiros Sintomas / Diagnóstico', 
                   'SEM_PRI': 'Semana Epidemiológica dos Primeiros Sintomas/Diagnósticos', 
                   'ANO_NASC': 'Ano de Nascimento', 
                   'NU_IDADE_N': 'Idade',
                   'CS_SEXO': 'Sexo', 
                   'CS_GESTANT': 'Gestação', 
                   'CS_RACA': 'Raça', 
                   'CS_ESCOL_N': 'Escolaridade', 
                   'SG_UF': 'UF', 
                   'ID_MN_RESI': 'Município de Residência',
                   'ID_RG_RESI': 'ID Município de Residência', 
                   'ID_PAIS': 'País (Se Residente Fora do Brasil)', 
                   'DT_INVEST': 'Data da Investigação', 
                   'ID_OCUPA_N': 'Ocupação', 
                   'ANT_DT_ACI': 'Data do Acidente',
                   'ANT_UF': 'UF', 
                   'ANT_MUNIC_': 'Município de Ocorrência do Acidente', 
                   'ANT_LOCALI': 'Localidade da Ocorrência do Acidente', 
                   'ANT_TEMPO_': 'Tempo Decorrido Picada/Atendimento', 
                   'ANT_LOCA_1': 'Local da Picada ',
                   'MCLI_LOCAL': 'Manifestações Locais', 
                   'CLI_DOR': 'Se Manifestações Locais Sim, Especificar (Dor)', 
                   'CLI_EDEMA': 'Se Manifestações Locais Sim, Especificar (Edema)', 
                   'CLI_EQUIMO': 'Se Manifestações Locais Sim, Especificar (Equimose)', 
                   'CLI_NECROS': 'Se Manifestações Locais Sim, Especificar (Necrose)',
                   'CLI_LOCAL_': 'Se Manifestações Locais Sim, Especificar (Outras)', 
                   'CLI_LOCA_1': 'No Caso de Outras(Especificar)', 
                   'MCLI_SIST': 'Manifestações Sistêmicas ', 
                   'CLI_NEURO': 'Se Manifestações Sistêmicas Sim, Especificar Neuroparalíticas (Ptose Palpebral, Turvação Visual)', 
                   'CLI_HEMORR': 'Se Manifestações Sistêmicas Sim, Especificar Hemorrágicas (Gengivorragia, Outros Sangramentos)',
                   'CLI_VAGAIS': 'Se Manifestações Sistêmicas Sim, Especificar Vagais (Vômitos/Diarreia)',
                   'CLI_MIOLIT': 'Se Manifestações Sistêmicas Sim, Especificar Miolíticas/Hemolíticas (Mialgia, Anemia, Urina Escura)',
                   'CLI_RENAL': 'Se Manifestações Sistêmicas Sim, Especificar Renais (Oligúria/Anúria)',
                   'CLI_OUTR_2': 'Se Manifestações Sistêmicas Sim, Especificar (Outras)',
                   'CLI_OUTR_3': 'No caso de Outras (especificar)',
                   'CLI_TEMPO_': 'Tempo de Coagulação',
                   'TP_ACIDENT': 'Tipo de Acidente ',
                   'ANI_TIPO_1': 'No caso de Outros, Especificar',
                   'ANI_SERPEN': 'Serpente - Tipo de Acidente',
                   'ANI_ARANHA': 'Aranha - Tipo de Acidente',
                   'ANI_LAGART': 'Lagarta - Tipo de Acidente',
                   'TRA_CLASSI': 'Classificação do Caso',
                   'CON_SOROTE': 'Soroterapia',
                   'NU_AMPOLAS': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antibrotópico (SAB)',
                   'NU_AMPOL_1': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Anticrotálico (SAC)',
                   'NU_AMPOL_8': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antiaracnídico (SAAr)', 
                   'NU_AMPOL_6': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antibrotópico - Laquético (SABL)',
                   'NU_AMPOL_4': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antielapídico (SAEL)',
                   'NU_AMPO_7': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antiloxoscélico (SALox)', 
                   'NU_AMPO_5': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antibrotópico - Crotálico (SABC)',
                   'NU_AMPOL_9': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antiescorpiônico (SAEsc)', 
                   'NU_AMPOL_3': 'Se Soroterapia Sim, Especificar, Número de Ampolas de Soro: Antilonômico (SALon)', 
                   'COM_LOC': 'Complicações Locais', 
                   'COM_SECUND': 'Se Complicações Locais Sim, Especificar - Infecção Secundária', 
                   'COM_NECROS': 'Se Complicações Locais Sim, Especificar - Necrose Extensa',
                   'COM_COMPOR': 'Se Complicações Locais Sim, Especificar - Síndrome Comportamental',
                   'COM_DEFICT': 'Se Complicações Locais Sim, Especificar - Déficit Funcional',
                   'COM_APUTAC': 'Se Complicações Locais Sim, Especificar - Amputação',
                   'COM_SISTEM': 'Se Complicações Locais Sim, Especificar - Complicações Sistêmicas',
                   'COM_RENAL': 'Se Complicações Locais Sim, Especificar - Insuficiência Renal', 
                   'COM_EDEMA': 'Se Complicações Locais Sim, Especificar - Insuficiência Respiratória/ Edema Pulmonar Agudo',
                   'COM_SEPTIC': 'Se Complicações Locais Sim, Especificar - Septicemia',
                   'COM_CHOQUE': 'Se Complicações Locais Sim, Especificar - Choque',
                   'DOENCA_TRA': 'Acidente Relacionado ao Trabalho',
                   'EVOLUCAO': 'Evolução do Caso',
                   'DT_OBITO': 'Data do Óbito',
                   'DT_ENCERRA': 'Data do Encerramento ',
                   'DT_DIGITA': 'Data de Digitação ',
                   'ID_MUNICIP_NOME': 'Nome Municipio de Notificação',
                   'ID_MN_RESI_NOME': 'Nome Município de Residencia', 
                   'ANT_MUNIC_NOME': 'Nome Município de Ocorrência do Acidente ',
                   'ID_OCUPA_N_NOME': 'Nome da Ocupação', 
                   'SG_UF_NOME': 'Sigla UF',
                   'SG_UF_NOT_NOME': 'Nome UF de Notificação',
                   'ANT_UF_NOME': 'Nome UF' 
}

Now that we have the complete dictionary, we use the **.rename** command on our DataFrame, so we will have it organized in a more appropriate way.

In [ ]:
final_data = datas.rename(columns = columns_renamed)

# Export to CSV
Let's export our final DataFrame to Excel in case there is a need to view the data in the program. This operation takes some time.

In [ ]:
final_data.to_csv('sus_acidente_animais_peconhentos_dados.csv', index=False, sep=';')